In [36]:
 # Import libraries
import pandas as pd
import pickle as pk

# import libraries for api
import os
from dotenv import load_dotenv 
import requests
import json

# import libraries for vizualization
import panel as pn
pn.extension("plotly")
pn.extension(comms="ipywidgets")
import param
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from pprint import pprint



In [37]:
# Import additional libraries that need to be installed after the panel extensions.
import holoviews as hv
import hvplot.pandas

In [38]:
#from Thomas's code 

filename = "Zillow_data"
infile = open(filename,'rb')
new_dict = pk.load(infile)
infile.close()

z_for_sale_resp =new_dict.json()

current_zillow_df = pd.json_normalize(data=z_for_sale_resp["props"])

print('num of rows:', len(current_zillow_df))
print('num of columns:', len(current_zillow_df.columns))

current_zillow_df.head()

num of rows: 40
num of columns: 18


,bathrooms,propertyType,lotAreaValue,address,imgSrc,price,listingDateTime,listingStatus,zpid,longitude,latitude,lotAreaUnit,daysOnZillow,bedrooms,country,currency,livingArea,hasImage
0,2,SINGLE_FAMILY,1742.40,"318 W Ruscomb St, Philadelphia, PA 19120",https://photos.zillowstatic.com/fp/c528e4507f6...,150000,None,FOR_SALE,10428068,-75.130410,40.026787,sqft,-1,4,USA,USD,1440,True
1,2,SINGLE_FAMILY,3920.40,"4913 Castor Ave, Philadelphia, PA 19124",https://photos.zillowstatic.com/fp/84229141ec1...,325000,None,FOR_SALE,10291269,-75.091110,40.024803,sqft,-1,4,USA,USD,1920,True
2,4,SINGLE_FAMILY,0.43,"200 W Walnut Ln, Philadelphia, PA 19144",https://photos.zillowstatic.com/fp/b90b6b03745...,869000,None,FOR_SALE,2068504637,-75.181816,40.037050,acres,-1,5,USA,USD,3857,True
3,4,SINGLE_FAMILY,9583.20,"2320 N 50th St, Philadelphia, PA 19131",https://photos.zillowstatic.com/fp/458505836a3...,267000,None,FOR_SALE,10486635,-75.227104,39.996180,sqft,-1,5,USA,USD,4520,True
4,3,SINGLE_FAMILY,3049.20,"522 Gilham St, Philadelphia, PA 19111",https://photos.zillowstatic.com/fp/04cc332dd77...,144000,None,FOR_SALE,2071261604,-75.092926,40.050960,sqft,-1,5,USA,USD,2007,True


In [39]:
#thomas code

current_zillow_df.columns
### Select columns that we will need for the analysis
current_zillow_df.columns
current_price_df = current_zillow_df[[ "zpid","propertyType" , "price", "livingArea" , "bathrooms" , "bedrooms" ,"address", "listingStatus" , "latitude" , "longitude", "imgSrc"]].copy()
current_price_df.set_index("zpid", inplace=True)


#Pull only the zip code from the address column
current_price_df["address"] = current_price_df["address"].str[-5:]

# Change the column names to improve readability
current_price_df.rename(columns = {
        "livingArea" : "Area in sqft.",
        "propertyType" : "Type of Property",
        "address" : "zip code",
        "listingStatus" : "Status",
        "imgSrc" : "Image property"
        }, inplace = True)


In [40]:
#thomas code
filename = "Zillow_old_data"
infile = open(filename,'rb')
new_source = pk.load(infile)
infile.close()

z_recently_sold_resp =new_source.json()

sold_zillow_df = pd.json_normalize(data=z_recently_sold_resp)

print('num of rows:', len(sold_zillow_df))
print('num of columns:', len(sold_zillow_df.columns))
sold_zillow_df.columns

sold_price_df = sold_zillow_df[[ "zpid", "homeType", "lastSoldPrice", "livingArea" , "bathrooms" , "bedrooms" , "address.zipcode",  "homeStatus" , "latitude" , "longitude", "miniCardPhotos"]].copy()
sold_price_df.set_index("zpid", inplace=True)

# Change the column names to improve readability
sold_price_df.rename(columns = {
        "homeType" : "Type of Property",
        "lastSoldPrice" : "Price",
        "livingArea" : "Area in sqft.",
        "address.zipcode" :"zip code",
        "homeStatus" : "Status",
        "miniCardPhotos" : "Image property"
        }, inplace = True)

num of rows: 5
num of columns: 18


In [41]:
#Add new columns in dataframes for "price per sqft"
current_price_df["price_per_sqft"] = current_price_df["price"] / current_price_df["Area in sqft."]

sold_price_df["price_per_sqft"] = sold_price_df["Price"] / sold_price_df["Area in sqft."]



In [42]:
#Add a variable for the ZPID. Add a variable "Purchase_house" to be the ZPID of the house we are looking to buy. 
ZPID = '10285435'
Purchase_house = current_price_df.loc[[ZPID]]
Purchase_house

,Type of Property,price,Area in sqft.,bathrooms,bedrooms,zip code,Status,latitude,longitude,Image property,price_per_sqft
zpid,,,,,,,,,,,
10285435,SINGLE_FAMILY,458900,1984,3,4,19119,FOR_SALE,40.052326,-75.193275,https://photos.zillowstatic.com/fp/f5cca126d4e...,231.300403


In [43]:
# Add variables for the average price per square foot/bedrooms/bathrooms of the comparable properties
# This is based on properties that requier to be in similar already (only one zipcode in table sold_price_df)

sold_average_price_per_sqft = sold_price_df['price_per_sqft'].mean()
sold_average_sqft = sold_price_df['Area in sqft.'].mean()
sold_average_bathrooms = sold_price_df['bathrooms'].mean()
sold_average_bedrooms = sold_price_df['bedrooms'].mean()

In [44]:
# Add variables for the price per square foot/bedrooms/bathrooms of the home we are looking to buy
Purchase_house_price_per_sqft = round(Purchase_house.loc[ZPID,'price_per_sqft'],2)
Purchase_house_sqft = round(Purchase_house.loc[ZPID,'Area in sqft.'],2)
Purchase_house_bathrooms = round(Purchase_house.loc[ZPID,'bathrooms'],2)
Purchase_house_bedrooms = round(Purchase_house.loc[ZPID,'bedrooms'],2)

In [45]:
# Add variables that are the difference per square foot/bedrooms/bathrooms of the comparables price per sqft vs. the house we want to buy
difference_price_per_sqft = round((sold_average_price_per_sqft - Purchase_house_price_per_sqft),2)
difference_sqft = round((sold_average_sqft - Purchase_house_sqft),2)
difference_bathrooms = round((sold_average_bathrooms - Purchase_house_bathrooms), 2)
difference_bedrooms = round((sold_average_bedrooms - Purchase_house_bedrooms), 2)

In [46]:
# Add a variable for our estimated Fair Value of the home we are looking to buy ( comparables average price per sq ft * purchase home sq ft)
Calculated_Home_Value = round((sold_average_price_per_sqft *  Purchase_house_sqft))

In [47]:
# Add a function to return our recommendation 
def recommendation(): 
    if sold_average_price_per_sqft > Purchase_house_price_per_sqft:
        return(f"We estimate the fair value of this house to be ${Calculated_Home_Value}. This house is ${difference_sqft} per square foot less expensive than comparable properties. We recommend you buy this house.")
    else:
        return(f"We estimate the fair value of this house to be ${Calculated_Home_Value}. This house is ${difference_sqft} per square foot more expensive than comparable properties. We don't recommend you buy this house.")
recommendation()

recommendation_text = recommendation()
recommendation_text

"We estimate the fair value of this house to be $421773. This house is $248.6 per square foot more expensive than comparable properties. We don't recommend you buy this house."

In [48]:
# Add functions to return other statistics on sqft
def other_info_sqft(): 
    if sold_average_sqft > Purchase_house_sqft:
        return(f"The property you are looking to buy has {difference_sqft: .2f} fewer square feet than the average of the comparables.")
    elif sold_average_sqft == Purchase_house_sqft:
        return(f"The property you are looking to buy has the same amount of square feet as the average of the comparable properties.")
    else:
        return(f"The property you are looking to buy has {difference_sqft: .2f} more square feet than the average of the comparables.")

other_info_sqft_text = other_info_sqft()

In [49]:
# Add functions to return other statistics on bathrooms
def other_info_bathrooms(): 
    if sold_average_bathrooms > Purchase_house_bathrooms:
        return(f"The property you are looking to buy has {difference_bathrooms: .2f} fewer bathrooms than the average of the comparables.")
    elif sold_average_bathrooms == Purchase_house_bathrooms:
        return(f"The property you are looking to buy has the same amount of bathrooms as the average of the comparable properties.")
    else:
        return(f"The property you are looking to buy has {difference_bathrooms: .2f} more bathrooms than the average of the comparables.")

other_info_bathrooms_text = other_info_bathrooms()

In [50]:
# Add functions to return other statistics on bedrooms
def other_info_bedrooms(): 
    if sold_average_bathrooms > Purchase_house_bathrooms:
        return(f"The property you are looking to buy has {difference_bedrooms: .2f} more bedrooms than the average of the comparables.")
    elif difference_bedrooms == 0.0:
        return(f"The property you are looking to buy has the same amount of bedrooms as the average of the comparable properties.")
    else:
        return(f"The property you are looking to buy has {difference_bedrooms: .2f} fewer bedrooms than the average of the comparables.")

other_info_bedrooms_text = other_info_bedrooms()

In [51]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
from dash.dependencies import Input, Output

ModuleNotFoundError: No module named 'dash'

In [52]:
file=open('HouseForSale.html',"w")
file.write(recommendation_text)
file.close()

In [53]:
#test


#app.layout = html.Div[html.Div([ 
    #html.Div([
       # dcc.Input(
         #   id='ZID'.format('text'),
         #   type='text',
         #   placeholder="insert {}".format('text'),
          #  debounce=True,
         #   minLength=0, maxLength=50,
          #  disabled=False,
         #   readOnly=False,
           # required=False,
         #   size="20"
   #     ) 
 #   ,
#
        
 #   ])
#])]

In [54]:
#test

#dff = df.copy()
#dff = dff[dff['ZID'] == ZID]
#print("ZID:" + str(ZID)

In [70]:
zpid_list = current_zillow_df["zpid"].tolist()
zpid_list

['10428068',
 '10291269',
 '2068504637',
 '10486635',
 '2071261604',
 '10581168',
 '10344057',
 '10571315',
 '10522272',
 '10562210',
 '10530575',
 '10561841',
 '80947824',
 '10545517',
 '10543196',
 '10415768',
 '10285435',
 '10592206',
 '2067596161',
 '2111077001',
 '10534810',
 '10249709',
 '10486771',
 '10213970',
 '10563074',
 '10334072',
 '2067980164',
 '10267688',
 '10270130',
 '2067615925',
 '10273816',
 '10360239',
 '10290711',
 '10533805',
 '10215202',
 '165109684',
 '2092097137',
 '10533350',
 '10524437',
 '10313568']

In [87]:
# Define Panel Visualization Functions
## First vizualisation : comparison of caracteristics between current house for sale and mean house profile sold

def housing_caracteristics_comparison1():
    # Split caracteristics for each line of the plot
    caracteristics_1 = ["price per sqft", "Average sqft"]

    # Set the first figure
    fig1 = go.Figure(data=[
        go.Bar(name="Purchase House", x=caracteristics_1, y=[Purchase_house_price_per_sqft , Purchase_house_sqft]),
        go.Bar(name="Average House", x=caracteristics_1, y= [sold_average_price_per_sqft , sold_average_sqft ])
        ])

    # Split each figure in two subfigures
    fig1.update_layout(barmode='group', title = "Price and surface compared to average similar house")

     # return pane with the figure housing_comparatif_fig
    return  fig1


def housing_caracteristics_comparison2():
    caracteristics_2 = ["Bathrooms",  "Bedrooms"]
    
    # Set the second figure
    fig2 = go.Figure(data=[
        go.Bar(name="Purchase House", x=caracteristics_2, y=[ Purchase_house_bathrooms , Purchase_house_bedrooms]),
        go.Bar(name="Average House", x=caracteristics_2, y= [sold_average_bathrooms , sold_average_bedrooms ])
        ])
    
    # Split each figure in two subfigures
    fig2.update_layout(barmode='group', title = "Amount of rooms compared to average similar house")

    # create figure
    return  fig2    




# Define Widget that allows to choose a property
zpid_list = current_zillow_df["zpid"].tolist()
widget = pn.widgets.Select(name="Pick a ZPID from a house",options=zpid_list)
#widget = param.Selector(objects=zpid_list)

## Second vizualisation : bell curve for the sold house
@pn.depends(widget.param.value, watch=True)
def housing_price_rate(widget):
    # Set title with variable ZPID in function of the search
    title_content = f"Density curve for properties similar to property with ZPID :{widget}"

    # Define two plots: the desnity and the value of the current house for sale defined by the ZPID.
    denisty_curve = sold_price_df['price_per_sqft'].hvplot.kde(width=600, height = 400, title = title_content, hover_line_color="red", grid=True)
    vline = hv.VLine(Purchase_house_price_per_sqft , label="current value")

    # Join plots under a new variable name
    density_plt = denisty_curve*vline
    
    # return pane with the figure plot_units
    return density_plt






## Third vizualisation : prices of the different house by neighborhood and sort them
def housing_price_neighborhood():
    # Sort prices by mean price for each zip code in an descending order
    current_price_by_neighborhood = current_price_df.groupby("zip code").mean()
    current_price_by_neighborhood.reset_index(inplace=True)
    current_price_by_neighborhood.sort_values("price_per_sqft", ascending=False, inplace=True)

    # Plot image
   # housing_price_ranking = plt.figure()
    housing_price_ranking = current_price_by_neighborhood.hvplot(
        kind="bar",
        x="zip code",
        xlabel = "zip code",
        y= "price_per_sqft",
        ylabel = " Price per sqft",
        title=" Price houses for sale by neighborhood",
        figsize =(10, 5),
        rot = 90
        )
   # plt.close(housing_price_ranking)
    
    # return pane with the figure plot_units
    return housing_price_ranking


## Fourth vizualisation: price versius area 
def price_per_neighborhood():
    
    fig_scatter = px.scatter(
        current_price_df,
        x="Area in sqft.",
        y="price",
        opacity=0.65,
        color = "zip code",
        trendline= "lowess",
        trendline_scope= "overall",
        trendline_color_override= "darkblue",
        title = "Relation between price and size of the property for the overall market in Philladelphia",
        height = 600
        )
    pane = pn.pane.Plotly(fig_scatter)
    return pane

## Fifth vizualisation : Category plot for the houses, beds, bathrooms, price
def housing_category_analysis():
    plt_categories = px.parallel_coordinates(
        current_price_df,
        dimensions= [  'price_per_sqft',  "price", "bathrooms" , "bedrooms"],
        title = "Parallel Categories Analysis of houses for sale in Philadelphia",
        color=  "Area in sqft.",
        color_continuous_scale= px.colors.sequential.Inferno,
        height = 800)
    
    return plt_categories








In [90]:
 housing_price_rate(widget)

:Overlay
   .Distribution.I      :Distribution   [price_per_sqft]   (Density)
   .VLine.Current_value :VLine   [x,y]

In [88]:
## Panel of vizualiszations
# Create the columns for the tab layout (there are 5 different columns that show different figures)
image = Purchase_house["Image property"]

fist_column = pn.Column(
    "# Welcome to this calculator and analysis tool",
    "There are currently some properties for sale in zillow that may have some intresting caracteristics for a good investement. This dashboard will help you analyze quickly their caracteristics to make good decision for an investment.",
    " In this section you are able to choose one ZPID for which you would like to have some market information.",
    widget,
    housing_price_rate(widget.value),
    f"Property type : {zpid_list}" 
    )

second_column = pn.Column(
    "# Market general information through different neighborhoods",
    " A city is not an other, it is important to understand your investement market before making a decision. We suggest you to look at least at those two factors." ,
    "<strong> First factor </strong> : Ranking the neigborhoods in a price analysis. This graphs gives you a first impression of the average prices per neighborhood.",
    housing_price_neighborhood(),
    "<strong> Second factor </strong> : Analysis of the properties by size by neighborhood. This can help to understand the offer on the market.",
    price_per_neighborhood()
    )

third_column = pn.Column(
    "# House for sale compared to houses sold recently",
    housing_caracteristics_comparison1(),
    "<strong> Our analysis the size: </strong>",
    other_info_sqft(),
    housing_caracteristics_comparison2(),
    "<strong> Our analysis on rooms: </strong>",
    other_info_bathrooms(), 
    other_info_bedrooms(),
    "# Recommendation:",
    recommendation()
    
    )



last_column = pn.Column(
    "Search parmetres",
    "With this table you can remove the ZPID from the list of search that aren't intrested for you in order to keep a selection taillored for you."
    )

# Create the dashboard with the 5 columns
dashboard = pn.Tabs(
    ("Welcome" , fist_column),
    ("General market information" , second_column),
    ("Pro's and con's", third_column),
    ("detailled properties characteristics"),
    ("parameters" , last_column),
    dynamic=True
    )

# Create a Title for the Dashboard to finalize the Dashboard for publication.
dashboard_out = pn.Column(
    "# Real Estate analysis for Philadelphia",
    dashboard)



In [89]:
dashboard.servable()

BokehModel(combine_events=True, render_bundle={'docs_json': {'123fd84d-69bb-4a13-9a7f-7008d29407e0': {'defs': …